# Generating Word Clouds

In [ ]:
!pip install wordcloud

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import ast
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Used for querying the completed data from s3 -> check the html verison for the completed data outputs

# import boto3
# from botocore.client import Config
# import s3fs 
# import os
# mykey = ""
# mysecretkey = ""
# config = Config(connect_timeout=120, 
#                 retries={'max_attempts': 10})
# s3 = boto3.resource(
#     service_name='s3',
#     region_name='us-west-2',
#     aws_access_key_id=mykey,
#     aws_secret_access_key=mysecretkey
# )
# client = boto3.client(
#     service_name='s3',
#     region_name='us-west-2',
#     aws_access_key_id=mykey,
#     aws_secret_access_key=mysecretkey,
#     config=config
# )
# os.environ["AWS_DEFAULT_REGION"] = 'us-west-2'
# os.environ["AWS_ACCESS_KEY_ID"] = mykey
# os.environ["AWS_SECRET_ACCESS_KEY"] = mysecretkey

# bucket = 'uci-capstone-custfields'


In [ ]:
# Query the sample data to visualize the sample word clouds
custom_fields = pd.read_csv("../sample_data/sample_accounts_data/cleaned_accounts_sample.csv", 
                            usecols = ["tags","keywords","userdefinedfields","org_id","account_id"])

# Count Custom Fields

## Helper Function

In [ ]:
def sort_custom_count(custom_fields):
    custom_dict = defaultdict(int)
    for cfs in custom_fields:
        cfs_list = cfs.split(",")
        cfs_set = set()
        for k in cfs_list:
            cfs_set.add(k.strip())
        for k in cfs_set:
            custom_dict[k]+=1

    sorted_custom = [(k,v) for k, v in sorted(custom_dict.items(), key=lambda item: -item[1])]
    return sorted_custom

def sort_udf_count(user_defined_fields):
    custom_dict = defaultdict(int)
    for cfs in user_defined_fields:
        cfs_list = ast.literal_eval(cfs)
        for k in cfs_list.keys():
            custom_dict[k]+=1

    sorted_custom = [(k,v) for k, v in sorted(custom_dict.items(), key=lambda item: -item[1])]
    return sorted_custom

In [ ]:
def display_wordcloud(cust_field_tuple, title):
    cust_field_dict = dict()
    for k,v in cust_field_tuple:
        cust_field_dict[k] = v
    wc = WordCloud(collocations=False,background_color = "white",width=800, height=400).generate_from_frequencies(cust_field_dict)
    plt.figure(figsize = (20, 20), facecolor = None)
    plt.imshow(wc)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.savefig(f'../outputs/WordClouds/WordCloud_{title}.png')
    plt.show()

## Keywords

In [ ]:
keywords = custom_fields.keywords.apply(lambda x: x[1:-1] if type(x)==str else x)
keywords = keywords.dropna().tolist()
sorted_keywords = sort_custom_count(keywords)

In [ ]:
sorted_keywords[0:5]

In [ ]:
len(sorted_keywords)

In [ ]:
keywords_df=pd.DataFrame.from_records(sorted_keywords, columns = ["keyword","keyword_count"])
keywords_df.to_csv("../outputs/WordClouds/keywords_count.csv")
keywords_df

In [ ]:
display_wordcloud(sorted_keywords, title = "WordCloud_Keywords")

## Tags

In [ ]:
tags = custom_fields.tags.apply(lambda x: x[1:-1] if type(x)==str else x)
tags = tags.dropna().tolist()
sorted_tags = sort_custom_count(tags)

In [ ]:
sorted_tags[0:5]

In [ ]:
len(sorted_tags)

In [ ]:
tags_df=pd.DataFrame.from_records(sorted_tags, columns = ["tag","tag_count"])
tags_df.to_csv("../outputs/WordClouds/tags_count.csv")
tags_df

In [ ]:
display_wordcloud(sorted_tags, title = "WordCloud_Tags")

In [ ]:
tags_df = custom_fields[["org_id","tags"]].dropna()

In [ ]:
tags_df[tags_df.tags.str.contains("'WCUNLINK'")]["org_id"].unique()

## User Defined Fields

In [ ]:
userdefinedfields = custom_fields.userdefinedfields.dropna()
# userdefinedfields = userdefinedfields.dropna().tolist()
sorted_userdefinedfields = sort_udf_count(userdefinedfields)


In [ ]:
len(sorted_userdefinedfields)


In [ ]:
userdefinedfields_df=pd.DataFrame.from_records(sorted_userdefinedfields, columns = ["userdefinedfields","userdefinedfields_count"])
userdefinedfields_df.to_csv("../outputs/WordClouds/userdefinedfields_count.csv")
userdefinedfields_df

In [ ]:
display_wordcloud(sorted_userdefinedfields, title = "WordCloud_UserDefinedFields")